In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import timm
import torch
import os
from tqdm import tqdm

from torchvision import transforms
from utils import label_to_id, AverageMeter
from sklearn.metrics.pairwise import cosine_similarity
from torchmetrics import Accuracy

In [25]:
NUM_CLASSES = 50
device =  'cuda'
DATA_ROOT = "tiny-imagenet-200"

In [26]:
from models import Network
model = Network(NUM_CLASSES, emb_dim=256)
model.to(device)
model.load_state_dict(torch.load('weights/Triplet512end-end50_[27_12_18_45_51].h5'))
model.eval();

In [27]:
transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((128, 128)),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])

In [32]:
from sklearn.metrics import accuracy_score, top_k_accuracy_score

def test(path): 
    print("Testing ---- ", path)

    seen_seen_df = pd.read_csv(path).values

    pred_outs = []
    similarity_outs = []
    labels = []
    for row in tqdm(seen_seen_df):

        im1 = cv2.imread(os.path.join(DATA_ROOT, row[0]), cv2.IMREAD_COLOR)
        im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2RGB)
        tensor1 = transform(im1).unsqueeze(0).to(device)

        feature1 = model.forward_features(tensor1).cpu().squeeze()
        # feature1 = torch.flatten(feature1).squeeze()
        feature1 = torch.nn.functional.avg_pool2d(feature1, 4)
        feature1 = torch.nn.functional.normalize(feature1).squeeze()

        out1 = model(tensor1).cpu().squeeze().argmax()

        ########################################

        im2 = cv2.imread(os.path.join(DATA_ROOT, row[1]), cv2.IMREAD_COLOR)
        im2 = cv2.cvtColor(im2, cv2.COLOR_BGR2RGB)
        tensor2 = transform(im2).unsqueeze(0).to(device)

        feature2 = model.forward_features(tensor2).cpu().squeeze()
        # feature2 = torch.flatten(feature2).squeeze()
        feature2 = torch.nn.functional.avg_pool2d(feature2, 4)
        feature2 = torch.nn.functional.normalize(feature2).squeeze()

        out2 = model(tensor2).cpu().squeeze().argmax()

        #########################################

        pred_outs.append(1 if out1.item() == out2.item() else 0)
        labels.append(row[2])

        csim = torch.nn.functional.cosine_similarity(feature1, feature2, dim=0).item()
        similarity_outs.append(1 if csim > 0.1 else 0)

    
    print("Multiclass Accuracy: ", accuracy_score(labels, pred_outs))
    print("Cosine Similarity (0.1) Accuracy: ", accuracy_score(labels, similarity_outs))

In [34]:
from sklearn.metrics import accuracy_score, top_k_accuracy_score

def test_triplet(path): 
    print("Testing ---- ", path)

    seen_seen_df = pd.read_csv(path).values

    similarity_outs = []
    labels = []
    for row in tqdm(seen_seen_df):

        im1 = cv2.imread(os.path.join(DATA_ROOT, row[0]), cv2.IMREAD_COLOR)
        im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2RGB)
        tensor1 = transform(im1).unsqueeze(0).to(device)

        feature1 = model(tensor1).cpu()
        feature1 = torch.nn.functional.normalize(feature1).squeeze()

        ########################################

        im2 = cv2.imread(os.path.join(DATA_ROOT, row[1]), cv2.IMREAD_COLOR)
        im2 = cv2.cvtColor(im2, cv2.COLOR_BGR2RGB)
        tensor2 = transform(im2).unsqueeze(0).to(device)

        feature2 = model(tensor2).cpu()
        feature2 = torch.nn.functional.normalize(feature2).squeeze()


        #########################################
        labels.append(row[2])

        csim = torch.nn.functional.cosine_similarity(feature1, feature2, dim=0).item()
        similarity_outs.append(1 if csim > 0.1 else 0)

    
    print("Cosine Similarity (0.1) Accuracy: ", accuracy_score(labels, similarity_outs))

In [35]:
test_triplet('seen_seen_test.csv')
test_triplet('seen_unseen_test.csv')
test_triplet('unseen_unseen_test.csv')

Testing ----  seen_seen_test.csv


100%|██████████| 1000/1000 [01:31<00:00, 10.92it/s]


Cosine Similarity (0.1) Accuracy:  0.808
Testing ----  seen_unseen_test.csv


100%|██████████| 200/200 [00:18<00:00, 10.84it/s]


Cosine Similarity (0.1) Accuracy:  0.71
Testing ----  unseen_unseen_test.csv


100%|██████████| 200/200 [00:18<00:00, 10.79it/s]

Cosine Similarity (0.1) Accuracy:  0.775
